# 🔧 Turing Tumble Hybrid Neuro-Symbolic Agent
This notebook implements a hybrid neuro-symbolic pipeline for solving Turing Tumble puzzles. It includes:
- A vision-based parser to extract board states from images
- LLaVA for visual reasoning and descriptions
- Graph construction from parsed board states
- A Graph Neural Network (GNN) for symbolic reasoning and solving

In [ ]:
# 📦 Install required packages
!pip install transformers accelerate torch torchvision torchaudio
!pip install torch-geometric
!pip install networkx opencv-python pillow matplotlib

In [ ]:
# 📌 Step 1: Parse board state from image (simple mock parser)
import cv2
import numpy as np
def parse_board_state_from_image(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)
    # Placeholder detection
    board_state = {
        "components": [
            {"type": "ramp", "position": [2, 3]},
            {"type": "gear", "position": [4, 5]}
        ]
    }
    return board_state

In [ ]:
# 🧠 Step 2: Extract description using LLaVA
from transformers import LlavaProcessor, LlavaForConditionalGeneration
from PIL import Image
def extract_board_description_with_llava(image_path):
    processor = LlavaProcessor.from_pretrained("liuhaotian/llava-v1.5-7b")
    model = LlavaForConditionalGeneration.from_pretrained("liuhaotian/llava-v1.5-7b")
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt")
    output = model.generate(**inputs)
    caption = processor.decode(output[0], skip_special_tokens=True)
    return caption

In [ ]:
# 🔗 Step 3: Convert board state to graph
import networkx as nx
import torch
from torch_geometric.data import Data as GNNData
def convert_board_state_to_graph(board_state):
    G = nx.DiGraph()
    for idx, component in enumerate(board_state["components"]):
        G.add_node(idx, label=component["type"], position=component["position"])
    for i in range(len(board_state["components"]) - 1):
        G.add_edge(i, i + 1)
    return G

def convert_nx_to_gnn_data(graph):
    node_labels = [ord(graph.nodes[n]['label'][0]) for n in graph.nodes()]
    x = torch.tensor([[label] for label in node_labels], dtype=torch.float)
    edge_index = torch.tensor(list(graph.edges()), dtype=torch.long).t().contiguous()
    return GNNData(x=x, edge_index=edge_index)

In [ ]:
# 🧪 Step 4: Define GNN Model
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
class TuringTumbleGNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(1, 16)
        self.conv2 = GCNConv(16, 2)
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

In [ ]:
# 🚀 Step 5: Run Full Hybrid Pipeline
def hybrid_pipeline(image_path):
    print("[1] Extracting board description with LLaVA...")
    caption = extract_board_description_with_llava(image_path)
    print(f"[Caption]: {caption}")
    print("[2] Parsing board state from image...")
    board_state = parse_board_state_from_image(image_path)
    print("[3] Converting board state to graph...")
    nx_graph = convert_board_state_to_graph(board_state)
    gnn_data = convert_nx_to_gnn_data(nx_graph)
    print("[4] Running symbolic reasoning via GNN...")
    model = TuringTumbleGNN()
    output = model(gnn_data)
    print("[GNN Output]:", output)
    return output

In [ ]:
# 🔍 Test with Your Image (upload to Colab)
from google.colab import files
uploaded = files.upload()
for fname in uploaded.keys():
    print("Running hybrid pipeline on:", fname)
    hybrid_pipeline(fname)